# Setup

In [ ]:
# Teacher model for O3 Mini
teacher_model = 'o3-mini-2025-01-31'

# Basic Test: o3 to 4o Communication Loop

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = os.getenv("OPENAI_API_KEY"),
)

if not client.api_key:
    raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable.")

user_input = """ 
            You are Patricia, a knowledgeable and friendly customer support agent at Nissan of Hendersonville, a family-owned and operated Nissan dealership in Hendersonville, North Carolina. Your role is to assist customers with their inquiries, ensure they find the perfect Nissan vehicle to meet their needs, and provide a professional and friendly service experience. 

            Follow these guidelines while interacting with customers: 

            - If a message is received in a non-English language, respond in the same language.
            - Provide only information explicitly stated in this directive; do not guess or fabricate details.
            - Politely inform customers if any requested information is unavailable and suggest they call the dealership directly for verification.

            ## Primary Responsibilities

            - **Vehicle Assistance**: 
            - Help customers find suitable Nissan models within their budget.
            - Provide accurate details about Nissan models, features, pricing, and availability.
            - Use the `fetch_cars` function for up-to-date inventory information. Default missing filters with `-1` for numeric fields and an empty string for text fields.

            - **Appointment Scheduling**:
            - Collect and verify customer's full name, valid phone number, and email before scheduling any appointment.
            - Do not proceed with scheduling if any necessary information is missing.
            - Politely request missing details and explain their necessity for scheduling. If a customer refuses, courteously mention that scheduling is not possible without this information.
            - Never schedule on days the dealership is closed and respect business hours.

            - **Providing Reviews**:
            - For specific car interest or reviews, utilize the `find_car_review_videos` function to suggest YouTube review videos, requiring `car_make` and `car_model` parameters.

            - **Customer Experience**:
            - Lead with professionalism and enthusiasm, reflecting the family-owned business nature.
            - Assist with booking test drives, handling complaints, and answering car-buying questions.
            - Remember, never make assumptions about dealership staff or unknown dealership specifics.

            ## Time Awareness

            - Use the current EST time received to:
            - Determine dealership operating status (open/closed).
            - Schedule within business hours.
            - Provide information regarding the next opening times.
            - Factor the day of the week in availability discussions.
            - Avoid unnecessary mentions of hours unless inferred or directly asked.

            ## Closing Conversations

            - Conclude interactions with a clear closing message when the dialogue nears its end, such as:
            - "Thank you for chatting with me today. I'm glad I could help you with your questions about Nissan vehicles. Have a great day!"

            ## Information Reference (Verify before providing)

            - **Name**: Nissan of Hendersonville
            - **Address**: 1340 Spartanburg Hwy, Hendersonville, NC 28792
            - **Phone**: +1 (828) 697-2222
            - **Website**: https://www.nissanofhendersonville.com
            - **Business Hours**: Monday-Saturday: 9 AM - 7 PM; Sunday: Closed

            This approach ensures high-quality service and satisfaction while maintaining the integrity and credibility of the information provided.
            """
completion = client.chat.completions.create(
    model= "gpt-4o-mini-2024-07-18",
    messages= [
    { 'role': "system", 'content': "You are a corporate IT support expert." },
    { 'role': "user", 'content': user_input}
    ]
)

print(completion.choices[0].message.content)

# Generate o3-Mini Test & Evaulation Set

In [ ]:

# Define the helper function to generate a diverse user message by calling an LLM
def generate_diverse_user_message():
    """
    This function calls the LLM to generate a new question.
    The prompt instructs the LLM to assume the role of a new hire at a tech company
    who is having trouble with their machine and needs assistance, and then to ask a question.
    """
    generation_prompt = (
        "You are a new hire at a tech company. You are experiencing an issue with your work machine "
        "and need assistance solving the problem. Please ask a question to the IT support team "
        "about the issue you are facing. Be specific and provide enough context for them to understand. Just ask the question, "
    )
    
    # Call the LLM to get a new question based on the prompt
    generation_response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are a creative question generator."},
            {"role": "user", "content": generation_prompt}
        ]
    )
    
    # Return the newly generated user message
    return generation_response.choices[0].message.content

# Base system message to provide context for the corporate IT support expert
system_message = { "role": "system", "content": "You are a corporate IT support expert." }

# Container for storing completions
completions = []

# Generate 300 completions with diverse user messages
for i in range(300):
    # Generate a diverse user query for each iteration using the additional LLM call.
    diversified_message = generate_diverse_user_message()
    
    # Create a chat completion using the dynamic user query
    completion = client.chat.completions.create(
        model="o3-mini-2025-01-31",
        messages=[
            system_message,
            { "role": "user", "content": diversified_message }
        ],
        store=True,
        metadata={
            "model": "o3-mini-2025-01-31",
            "system": "IT support expert",
            "user": "IT user"
        }
    )
    
    # Get the generated response and store it
    response_text = completion.choices[0].message.content
    completions.append(response_text)
    
    # Print the iteration and the corresponding response from the LLM
    print(f"Completion {i+1}:\nUser Query: {diversified_message}\nResponse: {response_text}\n{'-'*60}")


# Checking Model Differences

In [ ]:
output_model = "gpt-4o-mini-2024-07-18"

user_input = """Hi IT support team, I'm having trouble with my work laptop running Windows 10. It frequently freezes and displays a blue screen error with the code "SYSTEM_SERVICE_EXCEPTION." This issue tends to occur when I try to run multiple applications simultaneously, especially when using our project management software. I've already tried restarting the computer and checking for updates, but the problem persists. Can you please assist me in diagnosing and resolving this issue? Thank you!"""

completion = client.chat.completions.create(
    model= output_model,
    messages= [
    { 'role': "system", 'content': "You are a corporate IT support expert." },
    { 'role': "user", 'content': user_input}
    ]
)

print(completion.choices[0].message.content)

In [ ]:
output_model = "ft:gpt-4o-mini-2024-07-18:personal::(obsecured)"

user_input = """Hi IT support team, I'm having trouble with my work laptop running Windows 10. It frequently freezes and displays a blue screen error with the code "SYSTEM_SERVICE_EXCEPTION." This issue tends to occur when I try to run multiple applications simultaneously, especially when using our project management software. I've already tried restarting the computer and checking for updates, but the problem persists. Can you please assist me in diagnosing and resolving this issue? Thank you!"""

completion = client.chat.completions.create(
    model= output_model,
    messages= [
    { 'role': "system", 'content': "You are a corporate IT support expert." },
    { 'role': "user", 'content': user_input}
    ]
)

print(completion.choices[0].message.content)

In [ ]:
output_model = "o3-mini-2025-01-31"

user_input = """Hi IT support team, I'm having trouble with my work laptop running Windows 10. It frequently freezes and displays a blue screen error with the code "SYSTEM_SERVICE_EXCEPTION." This issue tends to occur when I try to run multiple applications simultaneously, especially when using our project management software. I've already tried restarting the computer and checking for updates, but the problem persists. Can you please assist me in diagnosing and resolving this issue? Thank you!"""

completion = client.chat.completions.create(
    model= output_model,
    messages= [
    { 'role': "system", 'content': "You are a corporate IT support expert." },
    { 'role': "user", 'content': user_input}
    ]
)

print(completion.choices[0].message.content)

# Fully Fine Tuned

In [ ]:
output_model = "ft:gpt-4o-mini-2024-07-18:personal:it-support-03-04mini:(obsecured)"

user_input = """Hi IT support team, I'm having trouble with my work laptop running Windows 10. It frequently freezes and displays a blue screen error with the code "SYSTEM_SERVICE_EXCEPTION." This issue tends to occur when I try to run multiple applications simultaneously, especially when using our project management software. I've already tried restarting the computer and checking for updates, but the problem persists. Can you please assist me in diagnosing and resolving this issue? Thank you!"""

completion = client.chat.completions.create(
    model= output_model,
    messages= [
    { 'role': "system", 'content': "You are a corporate IT support expert." },
    { 'role': "user", 'content': user_input}
    ],
    temperature=1)

print(completion.choices[0].message.content)
